In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from plotting import imshow
import cv2


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
class AnalysisDataset(Dataset):
    def __init__(self, filenames, images_data, bin_count=50):
        self.filenames = filenames # image name
        self.images_data = images_data # images data: data used to construct the images 
        self.bin_count = bin_count # number of bins used 
        self.bins = np.linspace(0, 1, bin_count)  # Bin edges
        # we are creating bins between 0 and 1 in order to have the values of u and v

    def __len__(self):
        return len(self.filenames)
    
    def bin_labels(self, values):
        return np.digitize(values, self.bins) -1 # Map values to bin indices
                                                   # This function takes values and assigns each value to a bin index
                                                   # based on self.bins. It returns the index starting from 0

    def __getitem__(self, idx):
        # Creating image from the dataset
        img_data = self.images_data[idx]
        img = np.array(img_data, dtype=np.uint32)

        Y_channel = (img[0:1024].reshape(32, 32, 1) / 255).astype(np.float32)
        U_channel = (img[1024:2048].reshape(32, 32, 1) / 255).astype(np.float32)
        V_channel = (img[2048:].reshape(32, 32, 1) / 255).astype(np.float32)
        #return y, u, v, self.filenames[idx]

    
        #return Y_channel, U_channel, V_channel, self.filenames[idx]

        # Binned shapes are (H,W)
        U_binned = self.bin_labels(U_channel)  # Discretize U 
        V_binned = self.bin_labels(V_channel)  # Discretize V

        paired_UV = np.stack((U_binned, V_binned), axis=2)

        return Y_channel, U_binned, V_binned,paired_UV, self.filenames[idx]

In [ ]:
yuv_train_set = unpickle('yuv_train')

yuv_images_data = yuv_train_set[b'data']

filenames = [f.decode('utf-8') for f in yuv_train_set[b'filenames']]

yuv_dataset = AnalysisDataset(filenames=filenames, images_data=yuv_images_data)
yuv_train_loader = DataLoader(yuv_dataset, batch_size=32, shuffle = False)

In [ ]:
# Input tensor(C,H,W) -> output tensor(H,W,C)
def rgb_yuv(array):
    new_array = array.permute(1,2, 0).numpy().copy()
    return torch.Tensor(cv2.cvtColor(new_array, cv2.COLOR_RGB2YUV))

# Input tensor(H,W,C) -> output tensor(C,H,W)
def yuv_rgb(array):
    new_array = array.numpy().copy()
    new_array = cv2.cvtColor(new_array, cv2.COLOR_YUV2RGB)
    return torch.tensor((new_array*255).astype(np.uint8)).permute(2, 0, 1)

In [ ]:
dataset = AnalysisDataset(filenames=filenames, images_data=yuv_images_data)
train_loader = DataLoader(dataset, batch_size=32, shuffle = False)

In [ ]:
all_U_binned = []
all_V_binned = []
all_UV_binned = []
for batch in train_loader:
    image_tensor, filename = batch  # Extract data from the batch
    for idx, image_rgb in enumerate(image_tensor):

Show the bins in the UV map

In [11]:
def plot_uv_histogram(all_U_binned, all_V_binned, bin_count=50):
    """
    Creates a histogram of binned U and V values.
    """
    U_flat = all_U_binned.flatten()  # Flatten tensor
    V_flat = all_V_binned.flatten()

    # Compute 2D histogram
    #hist, x_edges, y_edges = np.histogram2d(U_flat, V_flat, bins=bin_count, range=[[0, bin_count], [0, bin_count]])
    bins_u, counts_u = np.unique(U_flat,return_counts=True)[0],np.unique(U_flat,return_counts=True)[1]
    bins_v, counts_v = np.unique(V_flat,return_counts=True)[0],np.unique(V_flat,return_counts=True)[1]

    # Plot heatmap
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].bar(bins_u, counts_u, color='blue')
    ax[0].set_xlabel("U Bins")
    ax[0].set_ylabel("Counts")
    ax[0].set_title("Histogram of U values")

    ax[1].bar(bins_v, counts_v, color='red')
    ax[1].set_xlabel("V Bins")
    ax[1].set_ylabel("Counts")
    ax[1].set_title("Histogram of V values")

    plt.show()
    return bins_u,counts_u,bins_v,counts_v

def plot_uv_pairs_histogram(all_UV_binned, bin_count=50):
    """
    Creates a histogram of binned U,V paired values.
    """
    print("Shape of UV flat:", all_UV_binned.shape)
    UV_flat = all_UV_binned.flatten()  # Flatten tensor
    

    # Compute 2D histogram
    #hist, x_edges, y_edges = np.histogram2d(U_flat, V_flat, bins=bin_count, range=[[0, bin_count], [0, bin_count]])
    bins_uv, counts_uv = np.unique(UV_flat,return_counts=True)[0],np.unique(UV_flat,return_counts=True)[1]

    # Plot heatmap
    plt.bar(bins_uv, counts_uv, color='purple')
    plt.xlabel("UV Bins")
    plt.ylabel("Counts")
    plt.title("Histogram of U values")

    plt.show()
    
# Example usage
plot_uv_pairs_histogram(all_UV_binned,bin_count=50)



AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Example usage

bins_u,counts_u,bins_v,counts_v = plot_uv_histogram_pytorch(all_U_binned, all_V_binned, bin_count=50)
max_U = np.unique(bins_u[np.argmax(counts_u)])
max_V = np.unique(bins_v[np.argmax(counts_v)])

In [ ]:
# Define the number of bins
num_bins = 50
Y_fixed=128

# Create binned U and V values
U_bins = np.linspace(-128, 127, num_bins)
V_bins = np.linspace(-128, 127, num_bins)

# Create an image grid for binned U and V values
UV_binned_grid = np.zeros((num_bins, num_bins, 3), dtype=np.uint8)

# Iterate over the binned U, V values and convert to RGB
for i, U in enumerate(U_bins):
    for j, V in enumerate(V_bins):
        # Convert YUV to BGR
        YUV_color = np.array([Y_fixed, U + 128, V + 128], dtype=np.uint8)
        BGR_color = cv2.cvtColor(YUV_color.reshape(1, 1, 3), cv2.COLOR_YUV2BGR)
        UV_binned_grid[j, i] = BGR_color  # Assign to image (swapping j, i)

# Display the binned UV color map
plt.figure(figsize=(6, 6))
plt.scatter(max_U,max_V,s=50, facecolors='none', edgecolors='black',label="Most frequent bin")
plt.imshow(UV_binned_grid, extent=[0, num_bins - 1, 0, num_bins - 1])
plt.xlabel("U (Cb) Channel")
plt.ylabel("V (Cr) Channel")
plt.title(f"UV Color Map (Y = 128) with {num_bins} Bins")
plt.legend()
plt.grid(False)

plt.show()

Creating the probability distribution

In [ ]:
#
bins_u, counts_u = np.unique(all_U_binned.flatten(),return_counts=True)[0],np.unique(all_U_binned.flatten(),return_counts=True)[1]
bins_v, counts_v = np.unique(all_V_binned.flatten(),return_counts=True)[0],np.unique(all_V_binned.flatten(),return_counts=True)[1]

total_pixel = np.sum(counts_u)
full_bins = np.arange(0, num_bins)

# Create a dictionary mapping existing bins to their counts
bin_count_dict_u = dict(zip(bins_u, counts_u))
bin_count_dict_v = dict(zip(bins_v, counts_v))

# Ensure all bins have a count (default to 0 for missing bins)
full_counts_u = np.array([bin_count_dict_u.get(b, 0) for b in full_bins])
full_counts_v = np.array([bin_count_dict_v.get(b, 0) for b in full_bins])

probs_u = full_counts_u / np.sum(full_counts_u)
probs_v = full_counts_v / np.sum(full_counts_v)

'''
def u_distrib(bin_count=50):
    return np.random.choice(full_bins, bin_count,p=probs_u)

def v_distrib(bin_count=50):
    return np.random.choice(full_bins, bin_count,p=probs_v)
'''

In [ ]:
probs_u

### Compute weights for U and V binned separately

In [ ]:
def compute_weights(prob_dist, lambda_param=0.5):
    """
    Compute class rebalancing weights based on the pixel color distribution.
    """
    Q = len(prob_dist)  # Number of quantized bins
    uniform_dist = torch.ones(Q) / Q  # Uniform distribution

    # Compute the weighted probability
    smoothed_prob = (1 - lambda_param) * prob_dist + lambda_param * uniform_dist
    weights = 1.0 / smoothed_prob  # Inverse probability
    weights /= weights.sum()  # Normalize
    
    return weights

def soft_encode(y_true, bins, sigma=5):
    """
    Perform soft encoding by distributing weights across the nearest neighbors.
    - y_true: Ground truth colors (Nx1)
    - bins: Discrete ab bins (Qx1)
    - sigma: Gaussian kernel standard deviation
    """
    N = y_true.shape[0]
    print("Y_true shape: ",y_true.shape)
    Q = bins.shape[0]
    print("Q shape: ", bins.shape)
    
    # Compute Euclidean distance between each pixel and bin
    dist = torch.cdist(y_true, bins)  # Shape: (N, Q)
    
    # Find the 5 nearest neighbors
    knn_indices = torch.topk(-dist, k=5, dim=1).indices  # (N, 5)
    
    # Compute Gaussian weights
    knn_dist = torch.gather(dist, 1, knn_indices)  # Distances to 5 nearest neighbors
    weights = torch.exp(-knn_dist ** 2 / (2 * sigma ** 2))
    weights /= weights.sum(dim=1, keepdim=True)  # Normalize
    
    # Create soft-encoded one-hot vectors manually
    soft_labels = torch.zeros(N, Q, device=y_true.device)
    
    for i in range(N):
        soft_labels[i, knn_indices[i]] += weights[i]
    
    return soft_labels



lambda_param = 0.5
weights = compute_weights(torch.Tensor(probs_u), lambda_param)

'''
# Fake color data (ground truth and bins)
y_true = torch.randn(num_samples, 2)  # Ground truth colors
bins = torch.randn(num_bins, 2)  # Quantized ab bins

# Soft encoding
soft_labels = soft_encode(y_true, bins, sigma=5)
'''


In [ ]:

whole_labels = []
for img_data in images_data:
    img_rgb = np.array(img_data, dtype=np.uint8).reshape(3, 32, 32)
    img_rgb = np.transpose(img_rgb, (1, 2, 0))  # Convert to HxWxC format

    # Convert RGB to LUV (YUV-like)
    image_yuv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Luv)
    
    # Normalize Y to [0,1]
    Y_channel = image_yuv[:, :, 0] / 255.0
    U_channel = torch.tensor(image_yuv[:, :, 1] / 255.0)
    V_channel = image_yuv[:, :, 2] / 255.0

    bins = torch.Tensor([np.linspace(0, 1, num_bins + 1),np.linspace(0, 1, num_bins + 1)]) 
    soft_labels = soft_encode(U_channel,bins, sigma=4)
    print("Shape soft labels: ",soft_labels.shape)
    #print("Max value: ",np.max(soft_labels))
    whole_labels.append(soft_labels)

### Compute weights for U and V binned together

In [ ]:
UV_binned_grid = np.zeros((num_bins, num_bins,2), dtype=np.uint8)

# Iterate over the binned U, V values and convert to RGB
for i, U in enumerate(U_bins):
    for j, V in enumerate(V_bins):
        UV_binned_grid[i, j] = np.array([U,V], dtype=np.uint8)

In [ ]:


def compute_weights(prob_dist, lambda_param=0.5):
    """
    Compute class rebalancing weights based on the pixel color distribution.
    """
    Q = len(prob_dist)**2  # Number of quantized bins
    uniform_dist = torch.ones(Q) / Q  # Uniform distribution

    # Compute the weighted probability
    smoothed_prob = (1 - lambda_param) * prob_dist + lambda_param * uniform_dist
    weights = 1.0 / smoothed_prob  # Inverse probability
    weights /= weights.sum()  # Normalize
    
    return weights

def soft_encode(y_true, bins, sigma=5):
    """
    Perform soft encoding by distributing weights across the nearest neighbors.
    - y_true: Ground truth colors (Nx2)
    - bins: Discrete ab bins (Qx2)
    - sigma: Gaussian kernel standard deviation
    """
    N = y_true.shape[0]
    print(N)
    Q = UV_binned_grid.shape[0]
    print(Q)
    
    # Compute Euclidean distance between each pixel and bin
    dist = torch.cdist(y_true, bins)  # Shape: (N, Q)
    
    # Find the 5 nearest neighbors
    knn_indices = torch.topk(-dist, k=5, dim=1).indices  # (N, 5)
    
    # Compute Gaussian weights
    knn_dist = torch.gather(dist, 1, knn_indices)  # Distances to 5 nearest neighbors
    weights = torch.exp(-knn_dist ** 2 / (2 * sigma ** 2))
    weights /= weights.sum(dim=1, keepdim=True)  # Normalize
    
    # Create soft-encoded one-hot vectors manually
    soft_labels = torch.zeros(N, Q, device=y_true.device)
    
    for i in range(N):
        soft_labels[i, knn_indices[i]] += weights[i]
    
    return soft_labels



lambda_param = 0.5
weights = compute_weights(torch.Tensor(probs_u), lambda_param)

'''
# Fake color data (ground truth and bins)
y_true = torch.randn(num_samples, 2)  # Ground truth colors
bins = torch.randn(num_bins, 2)  # Quantized ab bins

# Soft encoding
soft_labels = soft_encode(y_true, bins, sigma=5)
'''


In [ ]:

whole_labels = []
for img_data in images_data:
    img_rgb = np.array(img_data, dtype=np.uint8).reshape(3, 32, 32)
    img_rgb = np.transpose(img_rgb, (1, 2, 0))  # Convert to HxWxC format

    # Convert RGB to LUV (YUV-like)
    image_yuv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Luv)
    
    # Normalize Y to [0,1]
    Y_channel = image_yuv[:, :, 0] / 255.0
    U_channel = image_yuv[:, :, 1] / 255.0
    V_channel = image_yuv[:, :, 2] / 255.0

    bins = torch.Tensor([np.linspace(0, 1, num_bins),np.linspace(0, 1, num_bins)]).T

    uv_channels = torch.Tensor([U_channel,V_channel]).T

    soft_labels = soft_encode(uv_channels, bins, sigma=4)
    print("Shape soft labels: ",soft_labels.shape)
    #print("Max value: ",np.max(soft_labels))
    whole_labels.append(soft_labels)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        
        # Load MobileNetV2 pretrained model (use only convolutional layers)
        mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        self.encoder = mobilenet.features  # Feature extractor
        
        # Decoder (Lightweight CNN)
        self.decoder = nn.Sequential(
            nn.Conv2d(1280, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 2, kernel_size=3, padding=1),  # Output 2-channel (a, b)
            nn.Tanh()  # Normalize to [-1, 1] for LAB space
        )
    
    def forward(self, x):
        features = self.encoder(x)  # Extract features from grayscale input
        output = self.decoder(features)  # Predict (a, b) channels
        return output

# Example usage
if __name__ == "__main__":
    model = ColorizationModel()
    model.eval()
    sample_input = torch.randn(1, 1, 32, 32)  # Batch of 1 grayscale image (32x32)
    with torch.no_grad():
        output = model(sample_input)
    print("Output shape:", output.shape)  # Expected (1, 2, 32, 32)